In [27]:
import os
import numpy as np
import importlib

In [28]:
feature_number = 105

In [29]:
X = np.empty((0,feature_number))
y = np.empty((0,))
for dirname, _, filenames in os.walk('./result'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        file = np.load(file_path)
        X = np.concatenate((X, file['features']))
        y = np.concatenate((y, file['labels']))
y = y.astype(int)
print(X.shape)
print(y.shape)

(393345, 105)
(393345,)


In [30]:
import split

In [31]:
X, y = split.delete_question_mark(X, y)
print(X.shape)
print(y.shape)

(392309, 105)
(392309,)


In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled = scaler.fit_transform(X_train)

In [35]:
from sklearn.decomposition import PCA
pca = PCA(n_components= 0.999)
pca.fit(X_scaled)
X_pca = pca.transform(X_scaled)
X_pca.shape

(313847, 82)

In [36]:
X_W, y_W = split.split_wake_and_asleep(X_pca, y_train)
print(X_W.shape)
print(y_W.shape)

(313847, 82)
(313847,)


In [41]:
import balance
importlib.reload(balance)
X_new, y_new = balance.get_balance_class_downsample(X_W, y_W)
print(X_new.shape, y_new.shape)

(177548, 82) (177548,)


In [44]:
import intergrating_model

In [45]:
importlib.reload(intergrating_model)
W_model = intergrating_model.generate_model(X_W, y_W)

svm finished
log finished
NB finished
SVM: 0.9558069141309543
Logistic: 0.9559343635494663
Naive Bayes: 0.550007965588657


In [63]:
import test
import scoring
importlib.reload(scoring)
X_test_W, y_test_W = split.split_wake_and_asleep(X_test, y_test)
# test.test_test(X_test_W, y_test_W, pca, W_model)
X_test_W.shape

(78462, 105)

In [69]:
X_test_pca = pca.transform(X_test_W)
print(X_test_pca.shape)

(78462, 82)


In [91]:
import logistic_with_pytorch
importlib.reload(logistic_with_pytorch)
best_model = logistic_with_pytorch.train_logistic_model_pytorch(X_W, y_W, 1)
y_pred = logistic_with_pytorch.predict(best_model, X_test_pca)
y_pred = np.array(y_pred).flatten()
best_acc = np.mean(y_pred == y_test_W)
best_weight = 1

  0%|          | 0/2000 [00:00<?, ?it/s]

In [95]:
def frange(start, end, step):
    while start < end:
        yield start
        start += step

In [104]:
model = logistic_with_pytorch.train_logistic_model_pytorch(X_W, y_W, 100000)
y_pred = logistic_with_pytorch.predict(model, X_test_pca)
y_pred = np.array(y_pred)
acc = np.mean(y_pred == y_test_W)
print(acc)

  0%|          | 0/2000 [00:00<?, ?it/s]

0.7155438301343325


In [105]:
np.sum(y_pred == 0)

78462

In [100]:
for i in range(20, 100):
    model = logistic_with_pytorch.train_logistic_model_pytorch(X_W, y_W, i)
    y_pred = logistic_with_pytorch.predict(model, X_test_pca)
    y_pred = np.array(y_pred)
    acc = np.mean(y_pred == y_test_W)
    if acc > best_acc:
        best_weight = i
        best_acc = acc
        best_model = model
    print(f"weight={i}, acc={acc:.3f}")
        
print("best_weight:", i)

  0%|          | 0/2000 [00:00<?, ?it/s]

weight=20, acc=0.716


  0%|          | 0/2000 [00:00<?, ?it/s]

weight=21, acc=0.716


  0%|          | 0/2000 [00:00<?, ?it/s]

...

weight=27, acc=0.716


  0%|          | 0/2000 [00:00<?, ?it/s]

weight=28, acc=0.716


  0%|          | 0/2000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [74]:
np.sum(y_pred == 0)

78462

In [ ]:
X_REM, y_REM = split.split_eye_movement(X_pca, y_train)
REM_model = intergrating_model.generate_model(X_REM, y_REM)

In [ ]:
X_N, y_N = split.split_deep_sleep(X_pca, y_train)
N_model = intergrating_model.generate_model(X_N, y_N)

In [ ]:
X_12, y_12 = split.split_1_2(X_pca, y_train)
model_12 = intergrating_model.generate_model(X_12, y_12)

In [ ]:
X_34, y_34 = split.split_3_4(X_pca, y_train)
model_34 = intergrating_model.generate_model(X_34, y_34)

In [ ]:
importlib.reload(intergrating_model)
y_pred = intergrating_model.int_model(X_test, pca, W_model, REM_model, N_model, model_12, model_34)

In [ ]:
np.sum(y_pred == 0) + np.sum(y_pred == 5)

In [ ]:
y_pred.shape

In [ ]:
import scoring

In [ ]:
importlib.reload(scoring)
scoring.score(y_pred, y_test, 6)